In [ ]:
# Google Analytics Data

In [ ]:
from helpers.cu_dataset_reader import CuDatasetReader

council_to_search = 'burnside'
dataset = CuDatasetReader().search_dataset_id_from_council_name(council_to_search)
dataset_id = ''
council_name = ''
if dataset is None:
    print('dataset not found for ', council_to_search)
else:
    dataset_id = dataset['dataset_id']
    council_name = dataset['council_name']

print('Council Name ', council_name)
print('dataset id ', dataset_id)
    

In [ ]:
from data_retrieval.google_analytics_api_retrieval import GoogleAnalyticsFilterClause, PageDto
from data_retrieval.google_analytics_data import GoogleAnalyticsData
from datetime import date

from dtos.date_range_dto import DateRangeDto
from helpers.settings_helper import SettingsHelper
CLIENT_SECRETS_FILE_PATH = './credentials/credentials.json'
TOKEN_FILE_PATH = './credentials/token.json'
ga_data = GoogleAnalyticsData(CLIENT_SECRETS_FILE_PATH, TOKEN_FILE_PATH)
filter_clause = GoogleAnalyticsFilterClause()
# comment this line to set dataset id to fetch data for all councils
filter_clause.set_dataset_id(dataset_id)
filter_clause.set_date_range(DateRangeDto(start_date=date(2022,1,1), end_date=date(2023,1,1)))
page_size = SettingsHelper().get_google_analytics_page_size()
filter_clause.set_page_dto(PageDto(page_size, None))
run_id = ga_data.save_data(filter_clause=filter_clause)
print('run id ', run_id)

In [ ]:
from helpers.enums import DataModule
from helpers.file_helper import FileHelper
file_helper = FileHelper()
# run_id = "2024-04-16-22-58-36_16538064-419e-4e6b-8aa4-5aed4e4e1d95"
landing_page_df = file_helper.read_run_file(run_id, DataModule.LANDING_PAGE)
landing_page_df

In [ ]:
import pandas as pd
sa_community_data_gov_au_export_df = pd.read_csv("./data/cu_export_2024-04-16_Data.Gov.au_export.csv")
sa_community_data_gov_au_export_df.head()

In [ ]:
from data_transform.clean_landing_page import CleanLandingPage

clean_landing_page = CleanLandingPage()
processed_data_df = clean_landing_page.process_data(landing_page_df, sa_community_data_gov_au_export_df)
processed_data_df.head()

In [ ]:
# these records are problematic, they are found in google analytics, but not in sacommunity council based export 
# Check these records manually, why it is not available in sacommunity db
# One posible reason is that the record in sacommunity is invalid, the council name could be wrong
# Later, will try to automate on how to get the exact council name from selenium
data_available_in_google_analytics_but_not_in_sacommunity_df = processed_data_df[processed_data_df["is_record_available_in_sacommunity_db"] == False]
data_available_in_google_analytics_but_not_in_sacommunity_df

In [ ]:
landing_page_cleaned_df = processed_data_df[processed_data_df["is_record_available_in_sacommunity_db"] == True]
landing_page_cleaned_df